# Quick exploration of geovec, a pre-trained word embedding in gloVe format

#### Data file of the pretrained embedding is downloaded from these links:

download site & information https://osf.io/4uyeq/wiki/home/

h5 file = https://osf.io/4uyeq/

Medium article by authors: https://towardsdatascience.com/geovec-word-embeddings-for-geosciences-ac1e1e854e19

In [40]:
#conda install -c anaconda scikit-learn 
# !pip install h5py
#!conda install -c anaconda h5py 

In [1]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
#from sklearn.manifold import TSNE

In [11]:
import json
import h5py
import numpy as np

##### glove python is a dependency mentioend by the authors
##### DON'T DO THIS AS IT HAS AlL SORTS OF PROBLEMS ON MACOS AND WINDOWS

#!pip install glove_python

### Instead of using glove-python, we can just open the file this way

In [12]:
with h5py.File('geovec_300d_v1.h5', 'r') as f:
        v = np.zeros(f['vectors'].shape, f['vectors'].dtype)
        f['vectors'].read_direct(v)
        dct = f['dct'][()].tostring().decode('utf-8')
        dct = json.loads(dct)

In [13]:
#### number of words with vector embeddings
print("number of words with vector embeddings",len(v))
#### Length of vectors
print("number of vectors in each embedding",len(v[1]))

number of words with vector embeddings 411845
number of vectors in each embedding 300


In [14]:
print("number of words total",len(dct))
print("and what each word in dictionary looks like: ",list(dct.keys())[0:1][0],":",dct[list(dct.keys())[0:1][0]])

number of words total 411845
and what each word in dictionary looks like:  czk : 22760


In [15]:
print("embedding for acrodermatitis",v[dct[list(dct.keys())[0:1][0]]])

embedding for acrodermatitis [-1.30995736e-01 -4.28154431e-02  2.80044712e-02  2.24730801e-02
  1.08891524e-01  6.49804547e-02 -2.53431760e-02  1.98111311e-02
  7.63765946e-02  5.05149923e-02  7.23294541e-02 -1.54078543e-01
  5.51027525e-03  2.01065950e-02  3.81736420e-02  6.16133250e-02
 -3.13544199e-02 -5.44905961e-02  2.41423100e-02 -7.96810985e-02
 -1.03932992e-02  2.94023491e-02 -6.15653917e-02 -4.95412126e-02
 -4.76407111e-02 -8.11042357e-03 -5.03108166e-02  2.33988669e-02
  1.40077055e-01 -3.35434303e-02  1.31544890e-02  8.25414881e-02
 -2.48029325e-02  3.51205282e-02  1.45962080e-02  1.84752196e-02
 -5.32907806e-02 -3.62833925e-02  7.11244904e-03  9.25709605e-02
  9.25662965e-02  1.30731110e-02  6.71290420e-03 -6.73174635e-02
 -1.04899555e-01  1.13067016e-01 -1.97199620e-02  1.14358701e-01
  5.27347662e-02  9.46701765e-02 -3.02816592e-02  3.35829519e-02
  4.47546132e-02 -1.33828958e-02  1.27948329e-01  1.12814896e-01
  2.29229406e-02  6.14958629e-02  3.79348993e-02 -1.09325647e

In [16]:
list(dct.keys())[0:10]

['czk',
 'zabin',
 'sesan',
 'midge',
 'coenopteridales',
 'microdisplacement',
 'lootable',
 'kurpiowska',
 'nucleophilic',
 'imbramowice']

In [17]:
type(v)

numpy.ndarray

In [18]:
#v

### We'll combine the two files into a single dictionary to make some later things a bit easier. It isn't saved in thi format to save space according to the authors who trained it

In [19]:
def replaceIntsWithWords(vectors,dictionary):
    words = list(dct.keys())
    embeddings_dict = {}
    for word in words:
        embeddings_dict[word] = np.asarray(v[dictionary[word]][:],"float32")
    return embeddings_dict

In [20]:
new_vector_embedding = replaceIntsWithWords(v,dct)

In [22]:
new_vector_embedding["sandstone"]

array([ 0.03063519, -0.41190132, -0.01748249, -0.03562951, -0.44922313,
       -0.23300318,  0.22224723, -0.07885347,  0.02758265, -0.07660752,
       -0.39882764,  0.19357805, -0.03862172, -0.16392523, -0.08809105,
       -0.15323128, -0.17489772,  0.14711891,  0.10426124, -0.02329986,
        0.09977868,  0.23435533,  0.3037015 , -0.6652086 , -0.1779219 ,
        0.16997509, -0.0749488 , -0.4623773 , -0.18799299, -0.04015233,
       -0.1230348 , -0.324141  ,  0.32538635,  0.27751258, -0.04844321,
        0.04106978,  0.3250498 ,  0.4131149 ,  0.41197386,  0.05734786,
       -0.28493047,  0.05330373, -0.30320197, -0.6228066 ,  0.17232999,
        0.13726279,  0.02862607,  0.00987866, -0.22104132, -0.27486506,
        0.3875012 ,  0.05712619,  0.30534878, -0.17705044, -0.19142899,
       -0.1692242 , -0.3759296 ,  0.35963646, -0.02505949,  0.05952422,
        0.33059987,  0.18033917,  0.05865888,  0.1908917 ,  0.03191795,
       -0.3452011 , -0.4242747 ,  0.02242155, -0.08702113, -0.36

### Now let's play around a bit to find out what words are close to others in the embedding space

In [23]:
def find_closest_embeddings(embeddings_dict,embedding_vector):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.euclidean(embeddings_dict[word], embedding_vector))

In [24]:
len(new_vector_embedding.keys())

411845

In [25]:
spatial.distance.euclidean(new_vector_embedding['sandstone'], new_vector_embedding['granite'])

4.911217212677002

In [26]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["sandstone"])[0:10]

['sandstone',
 'siltstone',
 'mudstone',
 'arkosic',
 'conglomerate',
 'rothbach',
 'gröden',
 'sonta',
 'mudstones',
 'interbedded']

In [27]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["granite"])[0:10]

['granite',
 'granitic',
 'ebat',
 'tuaní',
 'kibe',
 'lavadores',
 'baerzhe',
 'daguzhai',
 'cooyerdoo',
 'qôrqut']

In [28]:
find_closest_embeddings(new_vector_embedding,new_vector_embedding["crossbedding"])[0:20]

['crossbedding',
 'lumi',
 'unrein',
 'superface',
 'boundinaged',
 'staretsky',
 'essikado',
 'gerlarch',
 'doucette',
 'szenleng',
 'rhune',
 'shihui',
 'gackowa',
 'lenticularity',
 'tenappera',
 'karlshafen',
 'håkjerringdjupet',
 'anorbital',
 'eludicate',
 'torom']

### Looks like this is a lot of non-English words?!!

### Let's try subtraction of pure vectors

In [35]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["stratum"] - new_vector_embedding["fault"]
)[:25])

['sagyeri', 'pregrowth', 'barzaloza', 'translatent', 'syntransform', 'dothan', 'superincumbent', 'corniola', 'adoudou', 'chingmei', 'březno', 'saberian', 'kulm', 'trentinara', 'flamy', 'chulca', 'setlaole', 'emborozu', 'ikpe', 'guxu', 'aib', 'sarsbukta', 'huilu', 'nunda', 'lecho']


In [37]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["marble"] - new_vector_embedding["heat"]
)[:25])

['marble', 'sedef', 'pentelic', 'supren', 'macael', 'yeosan', 'calanchiole', 'pentelikon', 'kemalpasa', 'dionysos', 'borba', 'alabashka', 'iscehisar', 'maniçoba', 'hekkelstrand', 'benou', 'calcsilicates', 'evenes', 'harsin', 'montepuez', 'ayhan', 'nanshu', 'sanchuan', 'belevi', 'bettadabidu']


In [38]:
print(find_closest_embeddings(new_vector_embedding,
    new_vector_embedding["boulder"] - new_vector_embedding["clay"]
)[:25])

['intratill', 'satitoa', 'boulder', 'montoz', 'laxe', 'kintradwell', 'unclimbed', 'settai', 'norber', 'alveolized', 'intertill', 'cipit', 'preexposure', 'kudaka', 'moutonnee', 'ventifacted', 'navabandar', 'sturgis', 'taufererberg', 'pineo', 'preexposed', 'mathu', 'runswick', 'spelonk', 'encierro']


### A blunt approach doesn't work so well